### This program is designed to find the shortest path and the fastest path (using Euclidean Distance as a heuristic function) for a robot starting from a designated start point on the grid to a designated end point. 

In [110]:
# The class Vertex is a data structure for vertices in an undirected graph.
class Vertex:
    
    # Vertex object construction and variable initialization.
    def __init__(self, label):
        self.label = label
        self.edge = {}
        self.layer = 0; 
        self.parent = []
        self.explored = False
        self.tried = False
        self.pred = None
    
    # Returns the number of neighbors this vertex has.
    def degree(self):
        return len(self.edge)
    
    # Returns the shortest distance of this vertex to the source vertex. 
    # An unreachable vertex from the source vertex will have distance 0. 
    def getLayer(self):
        return self.layer
    
    # Sets this vertex's distance from the source vertex with the given value.
    def setLayer(self, layer):
        self.layer = layer
    
    # Returns the label of this vertex. In this case is the grid position of the vertex, 
    # which is in the form of a tuple (row, column).
    def getLabel(self):
        return self.label
    
    # Returns whether if this vertex is already tried, which is essential for both the
    # planShortest() and quickPlan() algorithm. 
    def isTried(self):
        return self.tried
    
    # Sets this vertex as tried.
    def setTried(self):
        self.tried = True
    
    # Returns the list of parent vertices that this vertex has.
    def getParent(self):
        return self.parent
    
    # Adds the given vertex as part of this vertex's list of parents. 
    def addParent(self, vertex):
        self.parent.append(vertex)
    
    # Returns whether if this vertex if already explored, which is essential for the
    # bfsShortestPath() algorithm.
    def isExplored(self):
        return self.explored
    
    # Sets this vertex as explored.
    def setExplored(self):
        self.explored = True
    
    # Returns the neighbors of this vertex in the form of a dictionary, 
    # where the keys are the grid position of the neighbors' vertices and
    # the values are the neighbors' vertices.
    def getNeighbors(self):
        return self.edge
    
    # Adds an undirected edge from this vertex to the specified vertex.
    def addEdge(self, vertex):
        if vertex.label not in self.edge:
            self.edge[vertex.label] = vertex
            vertex.edge[self.label] = self

    # Returns the predecessor of this vertex, which is the single parent that is set 
    # for this vertex during the quickPlan() algorithm. 
    def getPredecessor(self):
        return self.pred
    
    # Sets the predecessor of this vertex as the specified predecessor. 
    def setPredecessor(self, pred):
        self.pred = pred
    
    # Prints information about this vertex for debugging purposes. 
    def info(self):
        print("Vertex Label: " + self.label)
        if self.degree == 0:
            print("   This vertex has no edges.")
        else:
            edges = []
            if self.getNeighbors() != None:
                for element in self.getNeighbors():
                    edges.append(element)
            print("   Neighbors of this vertex: " + str(edges))
        print("   Layer: " + str(self.layer))
        if(len(self.getParent()) > 0):
            p = []
            for parent in self.getParent():
                p.append(parent.getLabel())
            print("   Parent: " + str(p))
        else:
            print("   This vertex has no parent.")
        print("   Vertex's predecessor: " + str(self.getPredecessor().getLabel()))
        print("   Vertex explored status: " + self.isExplored())
        print("   Vertex tried status: " + self.isTried())
        
    # Function responsible for finding all the shortest paths from a source vertex to a
    # destination vertex by assigning layers and parents for each vertex.
    def bfsShortestPath(self,vertex):
        self.setExplored()
        self.setLayer(0)
        queue = []
        queue.append(self)
        while len(queue) != 0:
            v = queue.pop(0)
            neighbors = v.getNeighbors()
            if neighbors != None:
                for element in neighbors:
                    if not neighbors[element].isExplored():
                        neighbors[element].setExplored()
                        queue.append(neighbors[element])
                        neighbors[element].setLayer(v.getLayer() + 1)
                    elif neighbors[element].getLayer() < v.getLayer():
                        v.addParent(neighbors[element])
    

import numpy as np
import math

# The class RobotPath represents a grid presenting all the possible paths a robot can move
# in a given grid starting from a start point to an end point. To make the robot's movement 
# simpler, it is designed to only move in 4 directions: South(s), North(n), West(w), and East(e).
class RobotPath:
    
    # RobotPath object construction and variable initialization. 
    def __init__(self):
        self.start = None
        self.end = None
        self.obstacles = []
        self.matrix = (-1,-1)
        self.path = (-1,-1)

        
    # Gets a input that is in the form of a text file with a certain format. An example format is as such:
    #
    #     nrows 10 ncols 10
    #     start 0 2
    #     dest 4 2
    #     obstacles
    #     1 2
    #     1 3
    #     3 3 
    #     3 4 
    #
    # --> The number after 'nrows' is the number of rows to be created, and the number 
    #     after 'ncols' is the number of columns to be created in each row. 
    #
    # --> The first number after 'start' is the row of the start point of the robot in the grid,
    #     whereas the second number after 'start' is the column of the start point of the robot
    #     in the grid. 
    #
    # --> The first number after 'dest' is the row of the end point of the robot in the grid,
    #     whereas the second number after 'dest' is the column of the end point of the robot
    #     in the grid. 
    #
    # --> All the lines after the line with 'obstacles' represent all the obstacles' coordinates
    #     in the grid. Obstacles are points in the grid in which the robot cannot past through it. 
    #     All the first number represent the row of each obstacle and the second number represent
    #     the column of each obstacle. 
    #
    # This function takes in the text file name as an input and converts all the information into a grid and 
    # changes all the corresponding class attributes (variables)
    def readInput(self, filename):
        with open(filename, 'r') as info:
            for lines in info:
                if lines.startswith('n'):
                    n = lines.split()
                    self.matrix = [ [0]*int(n[3]) for i in range(int(n[1]))]
                    self.path = [ [0]*int(n[3]) for i in range(int(n[1]))]
                    
                elif lines.startswith('s'):
                    n = lines.split()
                    self.start = (int(n[1]), int(n[2]))
                    self.matrix[int(n[1])][int(n[2])] = 'S'
                elif lines.startswith('d'):
                    n = lines.split()
                    self.end = (int(n[1]), int(n[2]))
                    self.matrix[int(n[1])][int(n[2])] = 'D'
                elif lines.startswith('o'):
                    continue
                else:
                    n = lines.split()
                    self.obstacles.append((int(n[0]), int(n[1])))
                    self.matrix[int(n[0])][int(n[1])] = '*'
        
        for rows in range(len(self.path)):
            for column in range(len(self.path[rows])):
                if (rows, column) not in self.obstacles:
                    self.path[rows][column] = Vertex((rows, column))
        
        for rows in range(len(self.path)):
            for column in range(len(self.path[rows])):
                if self.path[rows][column] != 0:
                    if (not rows - 1 < 0) and (self.path[rows-1][column] != 0):
                        self.path[rows][column].addEdge(self.path[rows-1][column])
                    
                    if (not rows + 1 > len(self.path) - 1) and (self.path[rows+1][column] != 0):
                        self.path[rows][column].addEdge(self.path[rows+1][column])
                    
                    if (not column + 1 > len(self.path[rows]) - 1) and (self.path[rows][column + 1] != 0):
                        self.path[rows][column].addEdge(self.path[rows][column + 1])
                    
                    if (not column - 1 < 0) and (self.path[rows][column - 1] != 0):
                        self.path[rows][column].addEdge(self.path[rows][column - 1])
    
    # This function uses the input grid and identifies all the possible shortest paths the robot can move
    # from the start point to the end point. The length of a plan is determined by the number of movements
    # the robot made to reach the destination (end point). A plan with a minimal length is a shortest plan. 
    def planShortest(self):
        self.path[self.start[0]][self.start[1]].bfsShortestPath(self.path[self.end[0]][self.end[1]])
        if self.path[self.end[0]][self.end[1]].getLayer() == 0:
            return
        vertex_list = [self.path[self.end[0]][self.end[1]]]
        while len(vertex_list) > 0:
            vertex = vertex_list.pop(0)
            if len(vertex.getParent()) > 0:
                for parent in vertex.getParent():
                    if parent == self.path[self.start[0]][self.start[1]]:
                        break
                    label = self.getDifference(vertex.getLabel(), parent.getLabel(), 0)
                    if self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] == 0:
                        self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] = label
                    else:
                        if label not in self.matrix[parent.getLabel()[0]][parent.getLabel()[1]]:
                            self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] = self.sortDirection(self.matrix[parent.getLabel()[0]][parent.getLabel()[1]] + label)
                    if not parent.isTried():
                        parent.setTried()
                        vertex_list.append(parent)
                    
    # This function finds a single path the robot can make from the start point to the end point quickly. 
    # The plan made by this function is not necessarily the shortest plan. The objective of this function 
    # is to use the input grid and identify a plan using the following strategies:
    # 
    # (a) No Back-Paddling: the plan must not include directives that makes the robot move through the same 
    #                       location more than once. 
    #
    # (b) Predictive Selection: from each location l, the robot can make multiple directives (at most 4)
    #                           to move to different locations. This algorithm first considers a directive choice
    #                           and if by using that choice, it can find a path to the destination, it will 
    #                           output the plan; otherwise, it will consider a different choice (until there
    #                           are no more choices to consider). This algorithm considers the choices in a 
    #                           specific order, such that any choice that is closer to the destination point
    #                           in terms of Euclidean distance will be considered first. If there are two choices
    #                           with the same distance from the destination, the one with a lower row value will 
    #                           be considered first; if the row choices are also the same, then the one with a 
    #                           lower column value will be considered first. For instance, if for the 
    #                           location (3,3), there are four possible moves (e, w, n, s), which leads to 
    #                           choices C = {(3,4), (3,2), (2,3), (4,3)}. The choice (4,3) is closest to the 
    #                           destination D = (8,7) and the next choice closest to D is (3,4), and so on. 
    #                           Therefore, the algorithm will first consider to move south from (3,3) to find a 
    #                           plan; if no plan is found using that choice, then the algorithm will consider to
    #                           move east from (3,3) to look for a plan; and after that it will consider move west,
    #                           and finally move north (given the previous choices fail to found a plan). 
    #
    # (c) Quick Stop: the search for a plan must terminate as soon as a plan is found. 
    def quickPlan(self):
        start = self.path[self.start[0]][self.start[1]]
        path = []
        start.setTried()
        path.append(start.label)
        while start != self.path[self.end[0]][self.end[1]]:
            if len(start.getNeighbors()) == 0:
                return
            choices = {}
            for element in start.getNeighbors():
                if not self.path[element[0]][element[1]].isTried():
                    choices[element] = self.EuclideanDistance((element[0], element[1]),self.end)
            if len(choices) != 0:
                key = list(choices.keys())
                value = list(choices.values())
                sort = np.argsort(value)
                sorted_dict = {key[i]: value[i] for i in sort}
                newDict = self.compare(sorted_dict)
                self.path[list(newDict.keys())[0][0]][list(newDict.keys())[0][1]].setPredecessor(start)
                start = self.path[list(newDict.keys())[0][0]][list(newDict.keys())[0][1]]
                start.setTried()
                path.append(start.label)
            else:
                path.remove(start.label)
                start = start.getPredecessor()
                if start == self.path[self.start[0]][self.start[1]]:
                    return
        self.pathTracer(path,1)

    # Function that assigns the directives for the path found in quickPlan().
    def pathTracer(self, paths, option):
        for i in range(1 , len(paths) - 1):
                direction = self.getDifference(paths[i], paths[i+1], 1)
                if self.matrix[paths[i][0]][paths[i][1]] == 0:
                    self.matrix[paths[i][0]][paths[i][1]] = direction
                else:
                    if direction not in self.matrix[paths[i][0]][paths[i][1]]:
                        self.matrix[paths[i][0]][paths[i][1]] = self.sortDirection(self.matrix[paths[i][0]][paths[i][1]] + direction)
    
    # Function that returns a sorted dictionary based on values of the dictionary.
    def compare(self, dic):
        if len(dic) == 1:
            return dic
        points = list(dic.keys())
        distance = list(dic.values())
        for i in range(len(distance) - 1):
            if distance[i] == distance[i+1]:
                if points[i+1][0] < points[i][0]:
                    temp = points[i+1]
                    points[i+1] = points[i]
                    points[i] = temp
                elif points[i+1][0] == points[i][0]:
                    if points[i+1][1] < points[i][1]:
                        temp = points[i+1]
                        points[i+1] = points[i]
                        points[i] = temp
                else:
                    continue
        newDict = {}
        for j in range(len(points)):
            newDict[points[j]] = distance[j]
        return newDict
    
    # Function that computes the Euclidean distance between two given points. 
    def EuclideanDistance(self, tuple1, tuple2):
        return math.sqrt(pow(tuple1[0]-tuple2[0],2) + pow(tuple1[1]-tuple2[1],2))
    
    # Get the differences between two points in a path and outputs the corresponding directive. Since planShortest()
    # is a backward tracing method, whereas quickPlan() is a forward tracing method, there are two options for 
    # this function: option = 0 indicates planShortest() and option = 1 indicates quickPlan().
    def getDifference(self, tuple1, tuple2, option):
        if tuple1[0] > tuple2[0] and tuple1[1] == tuple2[1]:
            if option == 0:
                return 's'
            elif option == 1:
                return 'n'
        elif tuple1[0] < tuple2[0] and tuple1[1] == tuple2[1]:
            if option == 0:
                return 'n'
            elif option == 1:
                return 's'
        elif tuple1[0] == tuple2[0] and tuple1[1] > tuple2[1]:
            if option == 0:
                return 'e'
            elif option == 1:
                return 'w'
        elif tuple1[0] == tuple2[0] and tuple1[1] < tuple2[1]:
            if option == 0:
                return 'w'
            elif option == 1:
                return 'e'
        else:
            return
    
    # Sorts the directives based on a given order of importance: s n w e. 
    def sortDirection(self, directions):
        order = {'s':1, 'n':2, 'w':3,'e':4}
        word = list(directions)
        for i in range(len(word)):
            indexSmallest = i
            for j in range(i + 1, len(word)):
                if order[word[j]] < order[word[indexSmallest]]:
                    indexSmallest = j
            temp = word[i]
            word[i] = word[indexSmallest]
            word[indexSmallest] = temp
        direction = ""
        for char in word:
            direction = direction + char
        return direction
    
    # Prints out the status of the grid. 
    #
    #  '0' : position in grid that is not part of the path of plan
    #  'S' : position in grid that represents the starting point of the robot
    #  'D' : position in grid that represents the destination point of the robot
    #  '*' : position of obstacles
    #  's' : Robot moving South in path to destination 
    #  'n' : Robot moving North in path to destination 
    #  'w' : Robot moving West in path to destination 
    #  'e' : Robot moving East in path to destination 
    #
    # For example, a 10 x 10 grid that has just been initialized may look like this: 
    #   0   0   0   *   0   0   0   0   0   0
    #   0   0   S   *   0   0   0   0   0   0
    #   0   *   *   0   0   0   0   0   0   0
    #   0   0   0   0   0   0   0   0   0   0
    #   0   0   0   0   0   0   *   0   0   0
    #   0   0   0   0   0   0   *   0   0   0
    #   0   0   0   0   0   0   *   0   0   0
    #   0   0   0   0   0   0   *   0   0   0
    #   0   0   0   0   0   0   0   D   0   0
    #   0   0   0   0   0   0   0   0   0   0
    #
    # After calling planShortest() on the above grid, it now looks like this:
    #   0   0   0   *   0   0   0   0   0   0
    #   s   w   S   *   0   0   0   0   0   0
    #   s   *   *   0   0   0   0   0   0   0
    #   se  se  se  se  se  se  e   s   0   0
    #   se  se  se  se  se  s   *   s   0   0
    #   se  se  se  se  se  s   *   s   0   0
    #   se  se  se  se  se  s   *   s   0   0
    #   se  se  se  se  se  s   *   s   0   0
    #   e   e   e   e   e   e   e   D   0   0
    #   0   0   0   0   0   0   0   0   0   0
    #
    # On the other hand, after calling quickPlan() on the above grid, it now looks like this:
    #   0   0   0   *   0   0   0   0   0   0
    #   s   w   S   *   0   0   0   0   0   0
    #   s   *   *   0   0   0   0   0   0   0
    #   e   e   e   s   0   0   0   0   0   0
    #   0   0   0   e   s   0   *   0   0   0
    #   0   0   0   0   e   s   *   0   0   0
    #   0   0   0   0   0   s   *   0   0   0
    #   0   0   0   0   0   s   *   0   0   0
    #   0   0   0   0   0   e   e   D   0   0
    #   0   0   0   0   0   0   0   0   0   0
    def output(self): 
        for rows in range(len(self.matrix)):
            for column in range(len(self.matrix[rows])):
                print(f"{self.matrix[rows][column]:>5}", end="")
            print()
        print()
    
    # Function to compare the outcome of planShortest() and quickPlan() algorithm with the expected outcome 
    # that is present another text file called 'expectFile'.
    def testOutput(self, expectFile):
        with open(expectFile, 'r') as expect:
            for rows in range(len(self.matrix)):
                line = expect.readline().split()
                for column in range(len(self.matrix[rows])):
                    if str(self.matrix[rows][column]) != str(line[column]):
                        point = (rows,column)
                        print("Error at " + str(point) + " of grid.")
                        return
            print("All correct")
            return

In [113]:
r = RobotPath()
r.readInput('Grid1.txt')
r.output()


    0    0    0    *    0    0    0    0    0    0
    0    0    S    *    0    0    0    0    0    0
    0    *    *    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    *    0    0    0
    0    0    0    0    0    0    0    D    0    0
    0    0    0    0    0    0    0    0    0    0



In [114]:
r.quickPlan()
r.output()

    0    0    0    *    0    0    0    0    0    0
    s    w    S    *    0    0    0    0    0    0
    s    *    *    0    0    0    0    0    0    0
    e    e    e    s    0    0    0    0    0    0
    0    0    0    e    s    0    *    0    0    0
    0    0    0    0    e    s    *    0    0    0
    0    0    0    0    0    s    *    0    0    0
    0    0    0    0    0    s    *    0    0    0
    0    0    0    0    0    e    e    D    0    0
    0    0    0    0    0    0    0    0    0    0



In [83]:
r.testOutput("11short.txt")

Error at (3, 7) of grid.


In [108]:
for i in range(1,20):
    inputFile = "Grid" + str(i) + ".txt" 
    expectFile = str(i) + "quick.txt"
    r = RobotPath()
    r.readInput(inputFile)
    r.quickPlan()
    print(str(i) + ":")
    r.testOutput(expectFile)

1:
All correct
2:
All correct
3:
All correct
4:
All correct
5:
All correct
6:
All correct
7:
All correct
8:
All correct
9:
All correct
10:
All correct
11:
All correct
12:
All correct
13:
All correct
14:
All correct
15:
All correct
16:
All correct
17:
All correct
18:
All correct
19:
All correct


In [109]:
for i in range(1,20):
    inputFile = "Grid" + str(i) + ".txt" 
    expectFile = str(i) + "short.txt"
    r = RobotPath()
    r.readInput(inputFile)
    r.planShortest()
    print(str(i) + ":")
    r.testOutput(expectFile)

1:
All correct
2:
All correct
3:
All correct
4:
All correct
5:
All correct
6:
All correct
7:
All correct
8:
All correct
9:
All correct
10:
All correct
11:
All correct
12:
All correct
13:
All correct
14:
All correct
15:
All correct
16:
All correct
17:
All correct
18:
All correct
19:
All correct


In [15]:
p = {8:'a'}
g = p.copy()
print(g)

{8: 'a'}


In [16]:
d = ("Hello" + "%d" + ".txt") % 3
print(d)

Hello3.txt


In [46]:
hello = [2]
hello.append([3,3])
print(hello)

[2, [3, 3]]


In [106]:
r = RobotPath()
r.createGrid("graph.txt")

Enter the number of rows for the desired grid: 10
Enter the number of columns for the desired grid: 10
Enter the start point in the form of (x,y): (-1,9)
('(', '-', '1', ',', '9', ')')


ValueError: invalid literal for int() with base 10: '('